<a href="https://colab.research.google.com/github/natalie-ayers/Iraq-post-conflict-rebel-governance/blob/main/Iraq_EVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import geopandas as gpd
import ee
import geemap
import ast
from shapely import geometry
import time

## Iraq Shapefiles

In [2]:
iraq_shp_adm0_loc = 'irq_admbnda_adm0_cso_itos_20190603.shp'
iraq_shp_adm0 = gpd.read_file(iraq_shp_adm0_loc)
iraq_shp_adm0.head(2)

,Shape_Leng,Shape_Area,ADM0_EN,ADM0_AR,ADM0_PCODE,ADM0_REF,ADM0ALT1EN,ADM0ALT2EN,ADM0ALT1AR,ADM0ALT2AR,date,validOn,validTo,geometry
0,35.691174,42.141484,Iraq,العراق,IQ,None,None,None,None,None,2019-05-30,2019-06-03,None,"POLYGON ((42.84654 37.34800, 42.85154 37.34657..."


In [2]:
iraq_shp_adm3_loc = 'irq_admbnda_adm3_cso_20190603.shp'
iraq_shp_adm3 = gpd.read_file(iraq_shp_adm3_loc)

In [3]:
iraq_adm3_filt = iraq_shp_adm3.loc[:,("Shape_Area",'ADM3_EN','ADM3_PCODE',
                                    'ADM2_EN','ADM2_PCODE','ADM1_EN','ADM1_PCODE',
                                    'geometry')]

iraq_adm3_filt.head(2)

,Shape_Area,ADM3_EN,ADM3_PCODE,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,geometry
0,0.015754,Abi Gharaq,IQG07Q02N02,Al-Hilla,IQG07Q02,Babil,IQG07,"POLYGON ((44.36654 32.56190, 44.36466 32.55802..."
1,0.145883,Abu Dalf,IQG16Q01N02,Al-Daur,IQG16Q01,Salah Al-Din,IQG16,"POLYGON ((44.19124 34.77808, 44.21397 34.75600..."


## MODIS EVI

In [4]:
ee.Authenticate()

In [5]:
ee.Initialize(project='iraq-postconflict')

In [7]:
iraq_bounds_ee = geemap.geopandas_to_ee(iraq_shp_adm0)
#iraq_bounds_ee.getInfo()

Functions to calculate EVI for given areas over a desired date range:

In [6]:
# this requires mounting google drive
def process_extracted_csv(csv_file):
  # requires mounting drive
  full_csv_path = f"drive/MyDrive/{csv_file}"
  evi_csv = pd.read_csv(full_csv_path)
  # 294 admin 3 regions, so can have
  # about 8 months of data (300 * 8 * 2 ~ 5000)
  # per table exported
  print(f"Csv shape: {evi_csv.shape}")

  # process the raw csv to have date and friendly names
  evi_csv['date'] = evi_csv['system:index'].str.extract(r'(.*)(?=_)')
  evi_csv['date'] = pd.to_datetime(evi_csv['date'],format="%Y_%m_%d")
  evi_csv.rename(columns={'max':'max_evi','mean':'mean_evi'},inplace=True)
  evi_csv.drop(columns=['system:index','.geo'],inplace=True)

  # save this new version
  new_csv_path = f"drive/MyDrive/processed_csvs_ee/processed_{csv_file}"
  print(f"saving processed csv for dates {evi_csv['date'].min()} to {evi_csv['date'].max()} as {new_csv_path}")
  evi_csv.to_csv(new_csv_path,index=False)

  return


In [7]:
# calculate a range of dates given
def calc_date_range(date_min, date_max, months_at_time):
  if date_max == 'today':
    date_max = pd.to_datetime('today').date()
  elif type(date_max) == str:
    date_max = pd.to_datetime(date_max).date()
  else:
    print("Error: date_max must be a string in the form 'yyyy-mm-dd' or left unspecified to use today's date")
    return

  date_min = pd.to_datetime(date_min).date()

  print(f"Calculating EVI over areas for date range {date_min.strftime(format='%Y-%m-%d')} to {date_max.strftime(format='%Y-%m-%d')}")


  start_range_freq = months_at_time * pd.offsets.MonthBegin()
  date_range_start = pd.date_range(date_min,date_max, freq=start_range_freq)

  # for longer ranges, add an extra week to ensure land in the desired month
  if months_at_time > 5:
    weeks_for_delta = months_at_time * 4 + 1
  else:
    weeks_for_delta = months_at_time * 4
  date_ranges = []
  for date in date_range_start:
    end_date = date + pd.Timedelta(weeks=weeks_for_delta)
    end_date = end_date + pd.offsets.MonthEnd()
    date_ranges.append((date.strftime(format='%Y-%m-%d'), end_date.strftime(format='%Y-%m-%d')))

  return date_ranges, date_min.strftime(format='%Y-%m-%d'), date_max.strftime(format='%Y-%m-%d')

# function to process modis EVI values in batches by date
def calc_modis_evi(areas_gpd, date_min, date_max='today', months_at_time=8):
  # convert area gpd to earth engine geometries
  areas_ee = geemap.geopandas_to_ee(areas_gpd)

  # create reducer and calc vi function from areas
  reducer = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.max(), sharedInputs=True
  )

  def calc_vi_stats(image):
    reduced = image.reduceRegions(
      collection=areas_ee,
      reducer=reducer,
      scale=30
      )
    return reduced

  # create range of dates to use in ee.Filter.date
  date_ranges, date_min, date_max = calc_date_range(date_min, date_max, months_at_time)

  for start, end in date_ranges:
    print(f"Processing date range {start} to {end}")
    modis_evi = ee.ImageCollection('MODIS/061/MOD13Q1').\
                filter(ee.Filter.date(start, end)).\
                select('EVI')

    evi_stats = modis_evi.map(calc_vi_stats).flatten()

    raw_csv = f"exportEviStats_{start}_{end}"
    print(f"Exporting results csv file {raw_csv}.csv")
    # can't convert more than 5000 elements
    task = ee.batch.Export.table.toDrive(
        collection = evi_stats,
        description=raw_csv,
        fileFormat='CSV'
    )
    task.start()
    while (task.status()['state']=='READY')|\
          (task.status()['state']=='UNSUBMITTED')|\
          (task.status()['state']=='RUNNING'):
      print("Export status state:",task.status()['state'])
      time.sleep(30)

    if task.status()['state'] != 'COMPLETED':
      print(f"Export failed for date range {start} to {end}")
      return

    # sometimes finishes before fully recognized in saved location;
    # give an extra 10 seconds just to confirm
    print("Final 40 second delay to confirm csv is ready")
    time.sleep(40)

    print(f"Processing csv file for final cleaning and readability")
    raw_csv = f"{raw_csv}.csv"
    process_extracted_csv(raw_csv)


In [8]:
calc_modis_evi(iraq_adm3_filt, date_min='2024-01-01')

Calculating EVI over areas for date range 2024-01-01 to 2024-03-07
Processing date range 2024-01-01 to 2024-08-31
Exporting results csv file exportEviStats_2024-01-01_2024-08-31.csv
Export status state: READY
Final 40 second delay to confirm csv is ready
Processing csv file for final cleaning and readability
Csv shape: (882, 11)
saving processed csv for dates 2024-01-01 00:00:00 to 2024-02-02 00:00:00 as drive/MyDrive/processed_csvs_ee/processed_exportEviStats_2024-01-01_2024-08-31.csv


To run a single batch of dates manually and piecewise, can use the code below:

In [80]:
# global images, so no point using filterBounds to select only Iraq
modis_all = ee.ImageCollection('MODIS/061/MOD13Q1').\
              filter(ee.Filter.date('2024-01-01', '2024-01-31'))
modis_evi = modis_all.select('EVI')
# properties['system:index':'2019_01_01']

In [81]:
iraq_adm3_ee = geemap.geopandas_to_ee(iraq_adm3_filt)

reducer = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.max(), sharedInputs=True
)

def calc_vi_stats(image):
  reduced = image.reduceRegions(
    collection=iraq_adm3_ee,
    reducer=reducer,
    scale=30
    )
  return reduced

evi_stats = modis_evi.map(calc_vi_stats).flatten()
# test with single image to confirm calcs are the same as in csv output
#evi_stats = calc_vi_stats(modis_evi.first())

In [82]:
# can't convert more than 5000 elements
task = ee.batch.Export.table.toDrive(
    collection = evi_stats,
    description="exportEviStats",
    fileFormat='CSV'
)
task.start()

In [ ]:
# takes a very long time, and lose the index with the date
#evi_stats_gpd = geemap.ee_to_gdf(evi_stats)

To visualize the EVI values produced, can use the code below:

In [ ]:
iraq_bounds_ee = geemap.geopandas_to_ee(iraq_shp_adm0)

In [20]:
Map = geemap.Map()

# Load an image.
image = modis_evi.first()

# Define the visualization parameters.
palett = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']

vizParams = {
  'bands': ['EVI'],
  'min': -2000,
  'max': 10000,
  'palette':palett
}



# Center the map and display the image.
Map.setCenter(44.230744, 34.967605, 8)
Map.addLayer(image.clip(iraq_bounds_ee), vizParams, 'Test')

# Display the map
Map

Map(center=[34.967605, 44.230744], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…